In [143]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import gensim
from tensorflow.keras.preprocessing.sequence import pad_sequences


2024-01-23 15:28:40.837614: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df=pd.read_csv('/Users/srishtysuman/PycharmProjects/NaturalLanguageProcessing/quora-question-pairs/train.csv')
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [3]:
df=df.iloc[:2000]

In [4]:
def clean(text):
    if type(text)!=str or text=='' or pd.isnull(text):
        return ''
    text = text.lower()
    text = re.sub("\'s", " ", text) 
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("can't", "can not", text)
    text = re.sub("n't", " not ", text)
    text = re.sub("i'm", "i am", text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    text = re.sub("e\.g\.", " eg ", text, flags=re.IGNORECASE)
    text = re.sub("b\.g\.", " bg ", text, flags=re.IGNORECASE)
    text = re.sub("(\d+)(kK)", " \g<1>000 ", text)
    text = re.sub("e-mail", " email ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text, flags=re.IGNORECASE)
    text = re.sub("\(s\)", " ", text, flags=re.IGNORECASE)
    text = re.sub("[c-fC-F]\:\/", " disk ", text)
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r'\d+', '',text)
    text = re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', text)
    text = text.replace("?","")
    text = text.replace("(","")
    text = text.replace(")","")
    text = text.replace('"',"")
    text = text.replace(",","")
    text = text.replace("#","")   
    text = text.replace("-","")    
    text = text.replace("..","")
    text = text.replace("/","")
    text = text.replace("\\","")
    text = text.replace(":","")
    text = text.replace("the","") 
    text = text.split()
    
    return text

In [5]:
"Regular expressions can contain both special and ordinary characters.".split()

['Regular',
 'expressions',
 'can',
 'contain',
 'both',
 'special',
 'and',
 'ordinary',
 'characters.']

In [6]:
df["question1"]=df["question1"].apply(lambda x: clean(x))
df["question1"]

/var/folders/g9/56rlvw7s6w54xn7rhdmb0qlw0000gn/T/ipykernel_23047/4173126970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["question1"]=df["question1"].apply(lambda x: clean(x))


0       [what, is, step, by, step, guide, to, invest, ...
1       [what, is, story, of, kohinoor, kohinoor, diam...
2       [how, can, increase, speed, of, my, internet, ...
3       [why, am, mentally, very, lonely, how, can, so...
4       [which, one, dissolve, in, water, quikly, suga...
                              ...                        
1995    [am, visiting, sri, lanka, soonfor, days, how,...
1996    [what, are, some, good, examples, of, stanza, ...
1997       [which, cpu, is, better, th, gen, or, th, gen]
1998    [what, are, some, of, best, tourist, places, t...
1999    [what, are, differences, between, love, marria...
Name: question1, Length: 2000, dtype: object

In [7]:
df["question2"]=df["question2"].apply(lambda x: clean(x))
df["question2"]

/var/folders/g9/56rlvw7s6w54xn7rhdmb0qlw0000gn/T/ipykernel_23047/208015887.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["question2"]=df["question2"].apply(lambda x: clean(x))


0       [what, is, step, by, step, guide, to, invest, ...
1       [what, would, happen, if, indian, government, ...
2       [how, can, internet, speed, be, increased, by,...
3       [find, remainder, when, [math]^{}[math], is, d...
4          [which, fish, would, survive, in, salt, water]
                              ...                        
1995                    [do, indians, hate, sri, lankans]
1996              [what, are, some, good, ilocano, poems]
1997     [which, is, better, intel, th, gen, or, th, gen]
1998    [where, are, foremost, tourist, places, in, ch...
1999    [which, is, better, an, arranged, marriage, or...
Name: question2, Length: 2000, dtype: object

In [8]:
for i, r in df.iterrows():
    print(i, type(r[3]), r['qid1'], r['qid2'], r['question1'], r['question2'])
    break

0 <class 'list'> 1 2 ['what', 'is', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india'] ['what', 'is', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market']


/var/folders/g9/56rlvw7s6w54xn7rhdmb0qlw0000gn/T/ipykernel_23047/3747024536.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(i, type(r[3]), r['qid1'], r['qid2'], r['question1'], r['question2'])


In [9]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('/Users/srishtysuman/Downloads/GoogleNewsvectorsnegative300.bin', binary=True)
word2vec


In [10]:
word2vec.key_to_index

{'</s>': 0,
 'in': 1,
 'for': 2,
 'that': 3,
 'is': 4,
 'on': 5,
 '##': 6,
 'The': 7,
 'with': 8,
 'said': 9,
 'was': 10,
 'the': 11,
 'at': 12,
 'not': 13,
 'as': 14,
 'it': 15,
 'be': 16,
 'from': 17,
 'by': 18,
 'are': 19,
 'I': 20,
 'have': 21,
 'he': 22,
 'will': 23,
 'has': 24,
 '####': 25,
 'his': 26,
 'an': 27,
 'this': 28,
 'or': 29,
 'their': 30,
 'who': 31,
 'they': 32,
 'but': 33,
 '$': 34,
 'had': 35,
 'year': 36,
 'were': 37,
 'we': 38,
 'more': 39,
 '###': 40,
 'up': 41,
 'been': 42,
 'you': 43,
 'its': 44,
 'one': 45,
 'about': 46,
 'would': 47,
 'which': 48,
 'out': 49,
 'can': 50,
 'It': 51,
 'all': 52,
 'also': 53,
 'two': 54,
 'after': 55,
 'first': 56,
 'He': 57,
 'do': 58,
 'time': 59,
 'than': 60,
 'when': 61,
 'We': 62,
 'over': 63,
 'last': 64,
 'new': 65,
 'other': 66,
 'her': 67,
 'people': 68,
 'into': 69,
 'In': 70,
 'our': 71,
 'there': 72,
 'A': 73,
 'she': 74,
 'could': 75,
 'just': 76,
 'years': 77,
 'some': 78,
 'U.S.': 79,
 'three': 80,
 'million': 81

In [11]:
word2vec.index_to_key

['</s>',
 'in',
 'for',
 'that',
 'is',
 'on',
 '##',
 'The',
 'with',
 'said',
 'was',
 'the',
 'at',
 'not',
 'as',
 'it',
 'be',
 'from',
 'by',
 'are',
 'I',
 'have',
 'he',
 'will',
 'has',
 '####',
 'his',
 'an',
 'this',
 'or',
 'their',
 'who',
 'they',
 'but',
 '$',
 'had',
 'year',
 'were',
 'we',
 'more',
 '###',
 'up',
 'been',
 'you',
 'its',
 'one',
 'about',
 'would',
 'which',
 'out',
 'can',
 'It',
 'all',
 'also',
 'two',
 'after',
 'first',
 'He',
 'do',
 'time',
 'than',
 'when',
 'We',
 'over',
 'last',
 'new',
 'other',
 'her',
 'people',
 'into',
 'In',
 'our',
 'there',
 'A',
 'she',
 'could',
 'just',
 'years',
 'some',
 'U.S.',
 'three',
 'million',
 'them',
 'what',
 'But',
 'so',
 'no',
 'like',
 'if',
 'only',
 'percent',
 'get',
 'did',
 'him',
 'game',
 'back',
 'because',
 'now',
 '#.#',
 'before',
 'company',
 'any',
 'team',
 'against',
 'off',
 'This',
 'most',
 'made',
 'through',
 'make',
 'second',
 'state',
 'well',
 'day',
 'season',
 'says',
 'w

In [12]:
vocabulary=dict()
inverse_vocabulary = ['<unk>']

In [13]:
stop_words = stopwords.words('english')
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [14]:
count=0
for index,row in df.iterrows():
    print(index)
    # Iterate through the text of both questions of the row
    for question in ['question1', 'question2']:
        q2n = []  # q2n -> question numbers representation
        for word in row[question]:
            # Check for unwanted words
            if word in stop_words and word not in word2vec.index_to_key:
                continue
            if word not in vocabulary:
                count+=1
                vocabulary[word] = count
                q2n.append(count)
                inverse_vocabulary.append(word)
            else:
                q2n.append(vocabulary[word])
        df.at[index, question] = q2n 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:
vocabulary


{'what': 1,
 'is': 2,
 'step': 3,
 'by': 4,
 'guide': 5,
 'invest': 6,
 'in': 7,
 'share': 8,
 'market': 9,
 'india': 10,
 'story': 11,
 'kohinoor': 12,
 'diamond': 13,
 'would': 14,
 'happen': 15,
 'if': 16,
 'indian': 17,
 'government': 18,
 'stole': 19,
 'back': 20,
 'how': 21,
 'can': 22,
 'increase': 23,
 'speed': 24,
 'my': 25,
 'internet': 26,
 'connection': 27,
 'while': 28,
 'using': 29,
 'vpn': 30,
 'be': 31,
 'increased': 32,
 'hacking': 33,
 'through': 34,
 'dns': 35,
 'why': 36,
 'am': 37,
 'mentally': 38,
 'very': 39,
 'lonely': 40,
 'solve': 41,
 'it': 42,
 'find': 43,
 'remainder': 44,
 'when': 45,
 '[math]^{}[math]': 46,
 'divided': 47,
 'which': 48,
 'one': 49,
 'dissolve': 50,
 'water': 51,
 'quikly': 52,
 'sugar': 53,
 'salt': 54,
 'methane': 55,
 'carbon': 56,
 'di': 57,
 'oxide': 58,
 'fish': 59,
 'survive': 60,
 'astrology': 61,
 'capricorn': 62,
 'sun': 63,
 'cap': 64,
 'moon': 65,
 'rising.what': 66,
 'does': 67,
 'that': 68,
 'say': 69,
 'about': 70,
 'me': 71

In [16]:
inverse_vocabulary

['<unk>',
 'what',
 'is',
 'step',
 'by',
 'guide',
 'invest',
 'in',
 'share',
 'market',
 'india',
 'story',
 'kohinoor',
 'diamond',
 'would',
 'happen',
 'if',
 'indian',
 'government',
 'stole',
 'back',
 'how',
 'can',
 'increase',
 'speed',
 'my',
 'internet',
 'connection',
 'while',
 'using',
 'vpn',
 'be',
 'increased',
 'hacking',
 'through',
 'dns',
 'why',
 'am',
 'mentally',
 'very',
 'lonely',
 'solve',
 'it',
 'find',
 'remainder',
 'when',
 '[math]^{}[math]',
 'divided',
 'which',
 'one',
 'dissolve',
 'water',
 'quikly',
 'sugar',
 'salt',
 'methane',
 'carbon',
 'di',
 'oxide',
 'fish',
 'survive',
 'astrology',
 'capricorn',
 'sun',
 'cap',
 'moon',
 'rising.what',
 'does',
 'that',
 'say',
 'about',
 'me',
 'triple',
 'ascendant',
 'this',
 'should',
 'buy',
 'tiago',
 'keeps',
 'childern',
 'active',
 'far',
 'from',
 'phone',
 'video',
 'games',
 'good',
 'geologist',
 'do',
 'great',
 'you',
 'use',
 'シ',
 'instead',
 'し',
 'motorola',
 'company',
 'hack',
 'cha

In [17]:
count=0
for index,row in df.iterrows():
    # Iterate through the text of both questions of the row
    for question in ['question1', 'question2']:
        q2n = []  # q2n -> question numbers representation
        for word in row[question]:
            # Check for unwanted words
            if word in stop_words and word not in word2vec.index_to_key:
                continue
            if word not in vocabulary:
                count+=1
                vocabulary[word] = count
                q2n.append(count)
                inverse_vocabulary.append(word)
            else:
                q2n.append(vocabulary[word])
        df.at[index, question] = q2n 

In [18]:
vocabulary

{'what': 1,
 'is': 2,
 'step': 3,
 'by': 4,
 'guide': 5,
 'invest': 6,
 'in': 7,
 'share': 8,
 'market': 9,
 'india': 10,
 'story': 11,
 'kohinoor': 12,
 'diamond': 13,
 'would': 14,
 'happen': 15,
 'if': 16,
 'indian': 17,
 'government': 18,
 'stole': 19,
 'back': 20,
 'how': 21,
 'can': 22,
 'increase': 23,
 'speed': 24,
 'my': 25,
 'internet': 26,
 'connection': 27,
 'while': 28,
 'using': 29,
 'vpn': 30,
 'be': 31,
 'increased': 32,
 'hacking': 33,
 'through': 34,
 'dns': 35,
 'why': 36,
 'am': 37,
 'mentally': 38,
 'very': 39,
 'lonely': 40,
 'solve': 41,
 'it': 42,
 'find': 43,
 'remainder': 44,
 'when': 45,
 '[math]^{}[math]': 46,
 'divided': 47,
 'which': 48,
 'one': 49,
 'dissolve': 50,
 'water': 51,
 'quikly': 52,
 'sugar': 53,
 'salt': 54,
 'methane': 55,
 'carbon': 56,
 'di': 57,
 'oxide': 58,
 'fish': 59,
 'survive': 60,
 'astrology': 61,
 'capricorn': 62,
 'sun': 63,
 'cap': 64,
 'moon': 65,
 'rising.what': 66,
 'does': 67,
 'that': 68,
 'say': 69,
 'about': 70,
 'me': 71

In [19]:
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[1, 2, 3, 4, 3, 5, 6, 7, 8, 9, 7, 10]","[1, 2, 3, 4, 3, 5, 6, 7, 8, 9]",0
1,1,3,4,"[1, 2, 11, 12, 12, 13]","[1, 14, 15, 16, 17, 18, 19, 12, 12, 13, 20]",0
2,2,5,6,"[21, 22, 23, 24, 25, 26, 27, 28, 29, 30]","[21, 22, 26, 24, 31, 32, 4, 33, 34, 35]",0
3,3,7,8,"[36, 37, 38, 39, 40, 21, 22, 41, 42]","[43, 44, 45, 46, 2, 47, 4]",0
4,4,9,10,"[48, 49, 50, 7, 51, 52, 53, 54, 55, 56, 57, 58]","[48, 59, 14, 60, 7, 54, 51]",0
...,...,...,...,...,...,...
1995,1995,3969,3970,"[37, 1590, 6376, 6377, 6378, 975, 21, 22, 4632...","[88, 2870, 1241, 6376, 6380]",0
1996,1996,3971,3972,"[1, 108, 109, 86, 405, 6381, 6382]","[1, 108, 109, 86, 6383, 6382]",0
1997,1997,3973,3974,"[48, 6384, 2, 252, 665, 3671, 158, 665, 3671]","[48, 2, 252, 2702, 665, 3671, 158, 665, 3671]",0
1998,1998,3975,3976,"[1, 108, 109, 186, 2210, 2360, 2558, 7, 6385]","[208, 108, 6386, 2210, 2360, 7, 6385]",1


In [21]:
df=df[["question1", "question2", "is_duplicate"]]
df

,question1,question2,is_duplicate
0,"[1, 2, 3, 4, 3, 5, 6, 7, 8, 9, 7, 10]","[1, 2, 3, 4, 3, 5, 6, 7, 8, 9]",0
1,"[1, 2, 11, 12, 12, 13]","[1, 14, 15, 16, 17, 18, 19, 12, 12, 13, 20]",0
2,"[21, 22, 23, 24, 25, 26, 27, 28, 29, 30]","[21, 22, 26, 24, 31, 32, 4, 33, 34, 35]",0
3,"[36, 37, 38, 39, 40, 21, 22, 41, 42]","[43, 44, 45, 46, 2, 47, 4]",0
4,"[48, 49, 50, 7, 51, 52, 53, 54, 55, 56, 57, 58]","[48, 59, 14, 60, 7, 54, 51]",0
...,...,...,...
1995,"[37, 1590, 6376, 6377, 6378, 975, 21, 22, 4632...","[88, 2870, 1241, 6376, 6380]",0
1996,"[1, 108, 109, 86, 405, 6381, 6382]","[1, 108, 109, 86, 6383, 6382]",0
1997,"[48, 6384, 2, 252, 665, 3671, 158, 665, 3671]","[48, 2, 252, 2702, 665, 3671, 158, 665, 3671]",0
1998,"[1, 108, 109, 186, 2210, 2360, 2558, 7, 6385]","[208, 108, 6386, 2210, 2360, 7, 6385]",1


In [28]:
df.groupby('is_duplicate')['is_duplicate'].count()

is_duplicate
0    1258
1     742
Name: is_duplicate, dtype: int64

In [29]:
df

,question1,question2,is_duplicate
0,"[1, 2, 3, 4, 3, 5, 6, 7, 8, 9, 7, 10]","[1, 2, 3, 4, 3, 5, 6, 7, 8, 9]",0
1,"[1, 2, 11, 12, 12, 13]","[1, 14, 15, 16, 17, 18, 19, 12, 12, 13, 20]",0
2,"[21, 22, 23, 24, 25, 26, 27, 28, 29, 30]","[21, 22, 26, 24, 31, 32, 4, 33, 34, 35]",0
3,"[36, 37, 38, 39, 40, 21, 22, 41, 42]","[43, 44, 45, 46, 2, 47, 4]",0
4,"[48, 49, 50, 7, 51, 52, 53, 54, 55, 56, 57, 58]","[48, 59, 14, 60, 7, 54, 51]",0
...,...,...,...
1995,"[37, 1590, 6376, 6377, 6378, 975, 21, 22, 4632...","[88, 2870, 1241, 6376, 6380]",0
1996,"[1, 108, 109, 86, 405, 6381, 6382]","[1, 108, 109, 86, 6383, 6382]",0
1997,"[48, 6384, 2, 252, 665, 3671, 158, 665, 3671]","[48, 2, 252, 2702, 665, 3671, 158, 665, 3671]",0
1998,"[1, 108, 109, 186, 2210, 2360, 2558, 7, 6385]","[208, 108, 6386, 2210, 2360, 7, 6385]",1


In [34]:
(1*np.random.randn(len(vocabulary) + 1, embedding_dim)).shape

(12777, 300)

In [30]:
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)
embeddings

array([[-0.21797372, -0.12956846,  2.22123541, ...,  0.37661856,
        -0.44510144,  0.66789666],
       [-1.44437714,  0.5911614 ,  2.71148021, ...,  1.74245402,
         1.3933197 , -1.93834873],
       [ 0.09602047, -0.14836203, -1.0166647 , ...,  0.68760762,
         0.2510784 ,  1.12895129],
       ...,
       [-0.59683841,  3.08012088, -0.44392976, ..., -0.30182158,
         0.35016434,  0.03170793],
       [-2.13980068, -0.43151334, -0.93891209, ..., -0.11239714,
         0.77835972, -0.3916716 ],
       [ 0.54997073,  0.6612037 ,  0.78579329, ...,  0.80750176,
         1.25122427,  1.74916323]])

In [36]:
embeddings[0] = 0
embeddings

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.44437714,  0.5911614 ,  2.71148021, ...,  1.74245402,
         1.3933197 , -1.93834873],
       [ 0.09602047, -0.14836203, -1.0166647 , ...,  0.68760762,
         0.2510784 ,  1.12895129],
       ...,
       [-0.59683841,  3.08012088, -0.44392976, ..., -0.30182158,
         0.35016434,  0.03170793],
       [-2.13980068, -0.43151334, -0.93891209, ..., -0.11239714,
         0.77835972, -0.3916716 ],
       [ 0.54997073,  0.6612037 ,  0.78579329, ...,  0.80750176,
         1.25122427,  1.74916323]])

In [37]:
vocabulary.items()

dict_items([('what', 1), ('is', 2), ('step', 3), ('by', 4), ('guide', 5), ('invest', 6), ('in', 7), ('share', 8), ('market', 9), ('india', 10), ('story', 11), ('kohinoor', 12), ('diamond', 13), ('would', 14), ('happen', 15), ('if', 16), ('indian', 17), ('government', 18), ('stole', 19), ('back', 20), ('how', 21), ('can', 22), ('increase', 23), ('speed', 24), ('my', 25), ('internet', 26), ('connection', 27), ('while', 28), ('using', 29), ('vpn', 30), ('be', 31), ('increased', 32), ('hacking', 33), ('through', 34), ('dns', 35), ('why', 36), ('am', 37), ('mentally', 38), ('very', 39), ('lonely', 40), ('solve', 41), ('it', 42), ('find', 43), ('remainder', 44), ('when', 45), ('[math]^{}[math]', 46), ('divided', 47), ('which', 48), ('one', 49), ('dissolve', 50), ('water', 51), ('quikly', 52), ('sugar', 53), ('salt', 54), ('methane', 55), ('carbon', 56), ('di', 57), ('oxide', 58), ('fish', 59), ('survive', 60), ('astrology', 61), ('capricorn', 62), ('sun', 63), ('cap', 64), ('moon', 65), ('ri

In [38]:
word2vec.index_to_key

['</s>',
 'in',
 'for',
 'that',
 'is',
 'on',
 '##',
 'The',
 'with',
 'said',
 'was',
 'the',
 'at',
 'not',
 'as',
 'it',
 'be',
 'from',
 'by',
 'are',
 'I',
 'have',
 'he',
 'will',
 'has',
 '####',
 'his',
 'an',
 'this',
 'or',
 'their',
 'who',
 'they',
 'but',
 '$',
 'had',
 'year',
 'were',
 'we',
 'more',
 '###',
 'up',
 'been',
 'you',
 'its',
 'one',
 'about',
 'would',
 'which',
 'out',
 'can',
 'It',
 'all',
 'also',
 'two',
 'after',
 'first',
 'He',
 'do',
 'time',
 'than',
 'when',
 'We',
 'over',
 'last',
 'new',
 'other',
 'her',
 'people',
 'into',
 'In',
 'our',
 'there',
 'A',
 'she',
 'could',
 'just',
 'years',
 'some',
 'U.S.',
 'three',
 'million',
 'them',
 'what',
 'But',
 'so',
 'no',
 'like',
 'if',
 'only',
 'percent',
 'get',
 'did',
 'him',
 'game',
 'back',
 'because',
 'now',
 '#.#',
 'before',
 'company',
 'any',
 'team',
 'against',
 'off',
 'This',
 'most',
 'made',
 'through',
 'make',
 'second',
 'state',
 'well',
 'day',
 'season',
 'says',
 'w

In [44]:
word2vec.word_vec("raised")

/var/folders/g9/56rlvw7s6w54xn7rhdmb0qlw0000gn/T/ipykernel_23047/181031725.py:1: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  word2vec.word_vec("raised")


array([-0.03686523, -0.04248047, -0.02258301,  0.04760742, -0.06152344,
       -0.18652344,  0.07470703, -0.03417969,  0.2109375 ,  0.14941406,
        0.02648926, -0.13085938,  0.0546875 , -0.0480957 ,  0.01721191,
        0.09082031, -0.06787109,  0.29492188, -0.42578125,  0.34375   ,
        0.06542969, -0.07519531, -0.02905273,  0.125     , -0.16210938,
       -0.15039062, -0.11523438, -0.09667969, -0.19726562, -0.04443359,
        0.16308594, -0.21972656, -0.11621094, -0.08203125, -0.04541016,
       -0.22753906, -0.15332031, -0.10791016,  0.04150391, -0.02172852,
       -0.04052734, -0.13574219, -0.07568359, -0.21289062, -0.33398438,
       -0.13378906, -0.15429688, -0.03686523,  0.19628906, -0.00066757,
        0.21972656, -0.22949219, -0.08300781, -0.08740234, -0.19921875,
        0.08886719, -0.2421875 ,  0.01226807, -0.05541992, -0.08984375,
        0.12353516,  0.09130859,  0.265625  , -0.3515625 , -0.16015625,
        0.02819824, -0.23242188, -0.05932617,  0.1328125 ,  0.06

In [45]:
# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.index_to_key:
        embeddings[index] = word2vec.word_vec(word)

del word2vec
embeddings

/var/folders/g9/56rlvw7s6w54xn7rhdmb0qlw0000gn/T/ipykernel_23047/1716599425.py:4: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  embeddings[index] = word2vec.word_vec(word)


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.13964844, -0.00616455,  0.21484375, ...,  0.05712891,
         0.09960938, -0.234375  ],
       [ 0.00704956, -0.07324219,  0.171875  , ...,  0.01123047,
         0.1640625 ,  0.10693359],
       ...,
       [-0.59683841,  3.08012088, -0.44392976, ..., -0.30182158,
         0.35016434,  0.03170793],
       [-2.13980068, -0.43151334, -0.93891209, ..., -0.11239714,
         0.77835972, -0.3916716 ],
       [ 0.54997073,  0.6612037 ,  0.78579329, ...,  0.80750176,
         1.25122427,  1.74916323]])

In [49]:
question1 = []
question2 = []
is_duplicate = []
for index,row in df.iterrows():
    print(row['question1'])
    print(row['question2'])
    print(row['is_duplicate'])
    break

[1, 2, 3, 4, 3, 5, 6, 7, 8, 9, 7, 10]
[1, 2, 3, 4, 3, 5, 6, 7, 8, 9]
0


In [51]:
max(df["question1"].map(lambda x: len(x)))

55

In [53]:
max_len=max(max(df["question1"].map(lambda x: len(x))), max(df["question2"].map(lambda x: len(x))))
max_len

55

In [ ]:
def pad_sequences( sequences, maxlen=None):
    num_samples = len(sequences)
    lengths = []
    sample_shape = ()
    flag = True

    for x in sequences:
        lengths.append(len(x))
        if flag and len(x):
            sample_shape = np.asarray(x).shape[1:]
            flag = False

    if maxlen is None:
        maxlen = np.max(lengths)

    x = np.full((num_samples, maxlen) + sample_shape, 0.0, dtype="int32")
    for idx, s in enumerate(sequences):
        trunc = s[-maxlen:]
        trunc = np.asarray(trunc, dtype="int32")
        x[idx, -len(trunc) :] = trunc

    return x

# q1_data = pad_sequences(df['question1'], maxlen=MAX_SEQUENCE_LENGTH)
# q2_data = pad_sequences(df['question2'], maxlen=MAX_SEQUENCE_LENGTH)
# labels = np.array(df['is_duplicate'], dtype=int)

In [123]:
def pad_sequence(sequence, max_len):
    if len(sequence)>max_len:
        return sequence[:max_len]
    len_x=len(sequence)
    return [0]*(max_len-len_x)+sequence

In [124]:
type(df["padded_q1"])

pandas.core.series.Series

In [125]:
MAX_SEQUENCE_LENGTH = 100
df["padded_q1"]=df["question1"].apply(lambda x: pad_sequence(x, MAX_SEQUENCE_LENGTH))
df["padded_q1"]

/var/folders/g9/56rlvw7s6w54xn7rhdmb0qlw0000gn/T/ipykernel_23047/816692514.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["padded_q1"]=df["question1"].apply(lambda x: pad_sequence(x, MAX_SEQUENCE_LENGTH))


0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
1995    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1996    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1997    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1998    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1999    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: padded_q1, Length: 2000, dtype: object

In [126]:
df["padded_q1"].iloc[0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 3,
 4,
 3,
 5,
 6,
 7,
 8,
 9,
 7,
 10]

In [127]:
df["padded_q1"]=df["question1"].apply(lambda x: pad_sequence(x, MAX_SEQUENCE_LENGTH))
df

/var/folders/g9/56rlvw7s6w54xn7rhdmb0qlw0000gn/T/ipykernel_23047/2432345105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["padded_q1"]=df["question1"].apply(lambda x: pad_sequence(x, MAX_SEQUENCE_LENGTH))


,question1,question2,is_duplicate,padded_q1,padded_q2
0,"[1, 2, 3, 4, 3, 5, 6, 7, 8, 9, 7, 10]","[1, 2, 3, 4, 3, 5, 6, 7, 8, 9]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[1, 2, 11, 12, 12, 13]","[1, 14, 15, 16, 17, 18, 19, 12, 12, 13, 20]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[21, 22, 23, 24, 25, 26, 27, 28, 29, 30]","[21, 22, 26, 24, 31, 32, 4, 33, 34, 35]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[36, 37, 38, 39, 40, 21, 22, 41, 42]","[43, 44, 45, 46, 2, 47, 4]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[48, 49, 50, 7, 51, 52, 53, 54, 55, 56, 57, 58]","[48, 59, 14, 60, 7, 54, 51]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
1995,"[37, 1590, 6376, 6377, 6378, 975, 21, 22, 4632...","[88, 2870, 1241, 6376, 6380]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1996,"[1, 108, 109, 86, 405, 6381, 6382]","[1, 108, 109, 86, 6383, 6382]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1997,"[48, 6384, 2, 252, 665, 3671, 158, 665, 3671]","[48, 2, 252, 2702, 665, 3671, 158, 665, 3671]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1998,"[1, 108, 109, 186, 2210, 2360, 2558, 7, 6385]","[208, 108, 6386, 2210, 2360, 7, 6385]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [128]:
df["padded_q2"]=df["question2"].apply(lambda x: pad_sequence(x, MAX_SEQUENCE_LENGTH))
df

/var/folders/g9/56rlvw7s6w54xn7rhdmb0qlw0000gn/T/ipykernel_23047/2504227262.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["padded_q2"]=df["question2"].apply(lambda x: pad_sequence(x, MAX_SEQUENCE_LENGTH))


,question1,question2,is_duplicate,padded_q1,padded_q2
0,"[1, 2, 3, 4, 3, 5, 6, 7, 8, 9, 7, 10]","[1, 2, 3, 4, 3, 5, 6, 7, 8, 9]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[1, 2, 11, 12, 12, 13]","[1, 14, 15, 16, 17, 18, 19, 12, 12, 13, 20]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[21, 22, 23, 24, 25, 26, 27, 28, 29, 30]","[21, 22, 26, 24, 31, 32, 4, 33, 34, 35]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[36, 37, 38, 39, 40, 21, 22, 41, 42]","[43, 44, 45, 46, 2, 47, 4]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[48, 49, 50, 7, 51, 52, 53, 54, 55, 56, 57, 58]","[48, 59, 14, 60, 7, 54, 51]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
1995,"[37, 1590, 6376, 6377, 6378, 975, 21, 22, 4632...","[88, 2870, 1241, 6376, 6380]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1996,"[1, 108, 109, 86, 405, 6381, 6382]","[1, 108, 109, 86, 6383, 6382]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1997,"[48, 6384, 2, 252, 665, 3671, 158, 665, 3671]","[48, 2, 252, 2702, 665, 3671, 158, 665, 3671]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1998,"[1, 108, 109, 186, 2210, 2360, 2558, 7, 6385]","[208, 108, 6386, 2210, 2360, 7, 6385]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [163]:
padded_q1=df["padded_q1"].values.tolist()
np.array(padded_q1)

array([[   0,    0,    0, ...,    9,    7,   10],
       [   0,    0,    0, ...,   12,   12,   13],
       [   0,    0,    0, ...,   28,   29,   30],
       ...,
       [   0,    0,    0, ...,  158,  665, 3671],
       [   0,    0,    0, ..., 2558,    7, 6385],
       [   0,    0,    0, ...,  153, 6388, 6387]])

In [165]:
padded_q2=df["padded_q2"].values.tolist()
np.array(padded_q2).shape

(2000, 100)

In [166]:
labels = np.array(df['is_duplicate'], dtype=int)
type(labels)
labels.shape

(2000,)

In [133]:
embeddings.shape

(12777, 300)

In [134]:
MAX_SEQUENCE_LENGTH = 100
SENT_EMBEDDING_DIM = 128
WORD_EMBEDDING_DIM = 300
NB_EPOCHS = 25
DROPOUT = 0.2
BATCH_SIZE = 516
VALIDATION_SPLIT = 0.1

In [167]:
q1_data, q2_data=padded_q1, padded_q2

In [168]:
y = labels
y

array([0, 0, 0, ..., 0, 1, 0])

In [172]:
X = np.stack((q1_data, q2_data), axis=1)
X.shape

(2000, 2, 100)

In [173]:
from sklearn.model_selection import train_test_split


In [174]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13371447)

In [175]:
Q1_train = X_train[:,0]
Q2_train = X_train[:,1]
Q1_test = X_test[:,0]
Q2_test = X_test[:,1]

In [176]:
y_train.shape

(1800,)

In [177]:
X_train

array([[[   0,    0,    0, ...,  220, 4459, 2841],
        [   0,    0,    0, ..., 2160, 4459, 2841]],

       [[   0,    0,    0, ...,    7, 1650, 1651],
        [   0,    0,    0, ...,    7, 1650, 1651]],

       [[   0,    0,    0, ...,  215, 4226, 1982],
        [   0,    0,    0, ...,  215, 4227, 1982]],

       ...,

       [[   0,    0,    0, ...,  183,  288, 5984],
        [   0,    0,    0, ..., 5984,   16,  530]],

       [[   0,    0,    0, ...,  288,   70,   71],
        [   0,    0,    0, ...,  183,  288,   71]],

       [[   0,    0,    0, ...,  901,  158,  902],
        [   0,    0,    0, ...,  902,  158,  203]]])

In [218]:
import torch

In [219]:
embeddings

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.13964844, -0.00616455,  0.21484375, ...,  0.05712891,
         0.09960938, -0.234375  ],
       [ 0.00704956, -0.07324219,  0.171875  , ...,  0.01123047,
         0.1640625 ,  0.10693359],
       ...,
       [-0.59683841,  3.08012088, -0.44392976, ..., -0.30182158,
         0.35016434,  0.03170793],
       [-2.13980068, -0.43151334, -0.93891209, ..., -0.11239714,
         0.77835972, -0.3916716 ],
       [ 0.54997073,  0.6612037 ,  0.78579329, ...,  0.80750176,
         1.25122427,  1.74916323]])

In [220]:
WORD_EMBEDDING_DIM

300

In [221]:
num_embeddings=len(embeddings)
num_embeddings

12777

In [222]:
import torch.nn as nn
import torch


In [223]:
# Model variables
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 25


In [ ]:
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))


In [ ]:
output_q1 = embeddings()

In [217]:
# The visible layer
q1 = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
q2 = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
type(q2)

keras.src.engine.keras_tensor.KerasTensor

In [ ]:




embedding_layer=torch.nn.Embedding(len(embeddings), embedding_dim, _freeze=False, weights=[embeddings])
embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=MAX_SEQUENCE_LENGTH, trainable=False)

# Embedded version of the inputs
encode_q1 = embedding_layer(q1)
encode_q2 = embedding_layer(q2)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

# Start training
training_start_time = time()

malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))